In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import scipy
import math

import scipy.stats as stats

import scipy.special as sps
from scipy.special import lambertw

from scipy import optimize
from scipy.stats import skew

In [19]:
def get_k(beta,xbar,x0):
    #k=kx, beta=<b>, x0=1/k
    def funct(k):
        return (1+(beta/x0))*k*beta/(1-(k*beta/x0))-xbar
    kopt=optimize.brentq(funct, 0, (x0/beta)-0.001)
    return kopt

def get_prot(g,t,p,p0):
    #p0=x0,#p=initial protein, t=time,g=gamma
    return np.real(p0*lambertw((p/p0)*np.exp(-g*t+(p/p0))))
    
def getder(g,t,p,p0):
    x=np.real(p0*lambertw((p/p0)*np.exp(-g*t+(p/p0))))
    return -g*x/(1+(x/p0))
# %%
class bacteria:
    def __init__(self,label,lamb1,ga,init_level,bs,timer):
        self.label=label
        self.lamb1=lamb1

        self.timer=timer
        self.ga=ga
        self.bs=bs
        self.prt_level=init_level#self.get_prtlevel()#Generate protein level
        
        self.nextt,self.reaction=self.get_nextt()
        self.prob=0.1
        
    def get_divt(self):
        r=np.random.rand()
        p0=self.prt_level
        def get_prots(t):
            p=get_prot(g=self.ga,t=t,p=p0,p0=x0)
            return p-p0*r
        def jac(t):
            dp=getder(g=self.ga,t=t,p=p0,p0=x0)
            return dp
            
        sol = optimize.root(get_prots, 0, jac=jac, method='hybr')
        td=sol.x[0]
        #print(td)
        return td

        
    def burst(self):

        bs_rst =np.random.exponential(self.bs)#-(self.bs)*np.log(np.random.rand())
        return bs_rst



    def get_nextt(self):
        # if self.state==0:
        r1=np.random.rand()
        t1=-(1/(self.lamb1))*np.log(r1)#protein production time
        if self.prt_level==0:
            r3=np.random.rand()
            t3=-(1/(self.ga))*np.log(r3)#division time
        else:
            t3=self.get_divt()#division time
        tau=np.min([t1,t3])
        react=np.argmin([t1,t3])
        
        return tau,react
        
    def divide(self):
        tot=self.prt_level
        xp2=self.prt_level
        return xp2
        
    


# %%
#script for simulating one population
dataf=[]
for x0 in [200]:#2*(10**np.linspace(1,3,10)):
    
    xbar=100
    bs=5
    lamb1=get_k(beta=bs,xbar=100,x0=x0)#kx


    ga=1 #growth rate
    grr=1/(1+(xbar/x0))
    tmax=2*(1/grr) #maximum time to simulation
    #print(tmax)
    dt=0.1 # time between samplings
    div=0
    t=0
    bs=1#birth size
    hill=1#hill exponent
  

    for colony in tqdm(range(100)):

        bct=bacteria(label=0,lamb1=lamb1,ga=ga,
                     init_level=xbar*np.random.gamma(shape=10,scale=0.1),bs=bs,timer=0)
        pop=[bct]

        nextt=pop[0].nextt #next reaction time
        react=pop[0].reaction #next reaction kind
        cellr=0 #next cell to reactaa

        dataf.append([x0,colony,bct.label,bct.prt_level,0,0,bct.ga]) #data sampling

        #for mother in (0,1000)
        t=0
        while t<=tmax:
            tt=dt
            if nextt<tt:
                while nextt<tt:
                    for cell in pop:
                        cell.timer+=nextt
                        cell.nextt+=-nextt#decreases the time to the next reaction for all cells by the time of the current reaction
                        cell.prt_level=get_prot(g=cell.ga,t=nextt,p=cell.prt_level,p0=x0)# np.exp(-cell.ga*nextt)#*(1+(cell.prt_level/k)**h))
                    tt+=-nextt
                    if react==0:
                        d_level=pop[cellr].burst()
                        pop[cellr].prt_level=d_level+pop[cellr].prt_level#get_prtlevel()
                    else:
                        pop[cellr].timer=0
                        xp2=pop[cellr].divide()
                        bct=bacteria(label=len(pop),lamb1=lamb1,ga=pop[cellr].ga,
                                     init_level= xp2,bs=bs,timer=0) #if the reaction was division, perform division
                        pop.append(bct)


                    pop[cellr].nextt,pop[cellr].reaction=pop[cellr].get_nextt()#re-calculate the next reaction for the cellr
                    tauarr=[]
                    reactarr=[]
                    for cell in pop:
                        tauarr.append(cell.nextt) #gathering the times of reaction for all cells
                        reactarr.append(cell.reaction) #gathering the kind of reaction for all cells
                    nextt=np.min(tauarr) #choosing the most inmediate reaction
                    cellr=np.argmin(tauarr) #choosing the cell of the most inmediate reaction
                    react=reactarr[cellr] #choosing the kind of reaction of the most inmediate reaction

                for cell in pop:
                    cell.timer+=tt
                    cell.prt_level=get_prot(g=cell.ga,t=tt,p=cell.prt_level,p0=x0)#np.exp(-cell.ga*tt)#*(1+(cell.prt_level/k)**h))
                    cell.nextt+=-tt
                nextt+=-tt #the remaining time before the next data sampling
                t+=dt #time increases a delta t
            else:
                nextt+=-dt #the sampling time is shorter than the closest reaction time
                t+=dt
                for cell in pop:
                    cell.timer+=dt
                    cell.nextt+=-dt #decreases the time to the next reaction for all cells by the time of the current reaction
                    cell.prt_level=get_prot(g=cell.ga,t=dt,p=cell.prt_level,p0=x0)#np.exp(-cell.ga*dt)#*(1+(cell.prt_level/k)**h))
        # question regarding this
            for cell in pop:

                dataf.append([x0,colony,cell.label,cell.prt_level,cell.timer,np.round(t,2),cell.ga]) #data sampling
df=pd.DataFrame(dataf,columns=['X0','Colony','Cell','Protein','Timer','Time','GrowthRate']) #comvert the array into dataframe
df.to_csv('./SingleCellDatax0.csv',index=False)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.60it/s]


In [20]:
df

,X0,Colony,Cell,Protein,Timer,Time,GrowthRate
0,200,0,0,53.872815,0.000000,0.0,1
1,200,0,0,49.943563,0.100000,0.1,1
2,200,0,0,47.134992,0.200000,0.2,1
3,200,0,0,44.005621,0.300000,0.3,1
4,200,0,0,40.518308,0.400000,0.4,1
...,...,...,...,...,...,...,...
13014,200,99,5,22.619470,0.712202,3.0,1
13015,200,99,6,19.391647,0.539553,3.0,1
13016,200,99,7,21.278234,0.163991,3.0,1
13017,200,99,8,20.027568,0.075361,3.0,1


In [21]:
dfcol=df[df.Colony==0]
for cell in dfcol.Cell.unique():
    datacell=dfcol[dfcol.Cell==cell]
    plt.plot(dfcell.Time,dfcell.Protein)

NameError: name 'dfcell' is not defined

In [17]:
pop[0].prt_level

11.037587509986393